# TTT4185 Machine learning for Speech technology

# Computer assignment 2: Classification using the Bayes Decision Rule and Support Vector Machines

This assignment assumes that the student has knowledge about the Bayes Decision Rule, maximum likelihood estimation and support vector machines.

In this assignment we will use `scikit-learn` (http://scikit-learn.org/stable/), which is a powerful and very popular Python toolkit for data analysis and machine learning, and `pandas` (https://pandas.pydata.org), which implements the all-powerful `DataFrame`.

We will also be using a small database of phonemes, where each phoneme is represented by the four first formant positions ("F1"-"F4") and their corresponding bandwidths ("B1"-"B4"). All numbers are in kHz. In addition, the speaker ID and the gender of the speaker are given for each phoneme.

In [ ]:
# this will fetch the Train.csv and Test.csv so you do not need to manually upload them. Just run this block for each session

! wget https://folk.ntnu.no/plparson/ttt4185_data/Train.csv
! wget https://folk.ntnu.no/plparson/ttt4185_data/Test.csv

## Problem 1: Maximum Likelihood

In this problem we will use the Bayes decision rule to classify vowels based on their formants. The formants have been extracted from the open database `VTR Formants database` (http://www.seas.ucla.edu/spapl/VTRFormants.html) created by Microsoft and UCLA.



### Task 1

Download the files `Train.csv` and `Test.csv` from Blackboard then upload them to your Colab workspace. You can load them into a `pandas` dataframe using the command `pd.read_csv`. Using the training data, create a single scatter plot of "F1" vs "F2" for the three vowels
- "ae" as in "bat"
- "ey" as in "bait"
- "ux" as in "boot"

Just eyeing the plots, discuss which classes will be hardest to classify correctly.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
train = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")

# Extract vowels
train = train[train['Phoneme'].isin(['ae', 'ey', 'ux'])]
test = test[test['Phoneme'].isin(['ae', 'ey', 'ux'])]

# Plotting here


### Task 2

 Use the Bayes Decision Rule to create a classifier for the phonemes 'ae', 'ey' and 'ux' under the following constraints:
- The feature vector $x$ contains the first two formants, "F1" and "F2".
- The distribution of $x$ given a phoneme $c$, $P(x|c)$, is Gaussian.
- Use the maximum likelihood estimator to estimate the model parameters.

### Task 3

To visualize the classes models and the classifier created in Task 2, plot the contours for each Gaussian distribution in the model. That is, the class conditional likelihoods $P(x|c)$, by using the following function.

In [ ]:
import scipy.stats

def plotGaussian(mean, cov, color, ax):
    """
        Creates a contour plot for a bi-variate normal distribution

        mean: numpy array 2x1 with mean vector
        cov: numpy array 2x2 with covarince matrix
        color: name of color for the plot (see https://matplotlib.org/stable/gallery/color/named_colors.html)
        ax: axis handle where the plot is drawn (can for example be returned by plt.gca() or plt.subplots())
    """
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    x, y = np.mgrid[xlim[0]:xlim[1]:(xlim[1]-xlim[0])/500.0, ylim[0]:ylim[1]:(ylim[1]-ylim[0])/500.0]
    xy = np.dstack((x, y))
    mvn = scipy.stats.multivariate_normal(mean, cov)
    lik = mvn.pdf(xy)
    ax.contour(x,y,lik,colors=color)

#### Optional

*Try:* Plot the decision regions for the Bayesian classifier. Tips: Calculate the posterior for each class, use the `numpy.argmax` function to get the decision regions, and `matplotlib.pyplot.contourf` to plot them.

### Task 4

Test your classifier on the 'ae', 'ey' and 'ux' phonemes from the test set and present your results in a _confusion matrix_. That is, a table where you see how many times 'ae' was correctly classified, how many times it was wrongly classified as 'ey' and so on.

### Task 5

Extend your classifier to include the features "F1"-"F4" and compare the results with those in Task 4.

### Task 6

Finally use all available information "F1"-"F4" and "B1-B4". How does the performance of this classifier compare with the simpler classifiers using fewer features?

### Task 7

To better understand how important each feature is to the model, we will now perform permutation feature importance (https://scikit-learn.org/stable/modules/permutation_importance.html). The idea is that for each feature we will randomly shuffle the values and the ask the trained model to predict using this shuffled data. We can then observe how the performance degregates as each feature is shuffled.

For the model, take one of the 8 features (four formants and four bandwidths), shuffle and evaluate it 10 times. Repeat this process for each feature. Which feature has the biggest impact on model performance? Does this align with your knowlege of formants?

Below we've provided a sample function to get you started

In [ ]:
def do_permutation_eval(classifier, df: pd.DataFrame, feature: str, num_iter: int) -> list:
  feat_results = []
  shuffle_df = df.copy(deep=True)
  # shuffle and evaluate multiple times to ensure the trends are real
  for i in range(num_iter):
    shuffled_values = list(shuffle_df[feature])
    # shuffle does this in-place
    np.random.shuffle(shuffled_values)
    shuffle_df[feature] = shuffled_values

    # This is psuedo-code
    # Modify to work with your own classifier implementations
    run_results = classifier.do_prediction(shuffle_df)
    run_accuracy = calculate_error_rate(run_results)
    feat_results.append(run_accuracy)
  return feat_results

### Task 8

We want to make the model slightly more powerful by modeling the feature vector conditional on both the vowel and gender of speaker, that is $P(x|g,c)$, where $g$ is the gender of the speaker and $c$ is the phoneme label. Show how these models can be used for phoneme classification using marginalization over the gender.

Assume that $P(x|g,c)$ is a multivariate Gaussian and compute the maximum likelihood estimates for the models. Compare the result on the test set with the results in Task 5.

### Task 9

When using Gaussian classifiers we often avoid computing the entire covariance matrix, but instead we only use the diagonal of the matrix. Repeat the results in Task 5 using only diagonal covariance matrices and compare the results.

## Problem 2: SVMs

In this problem we use the support vector machine (SVM) to build classifiers. We use the same dataset as in Problem 1. It is up to you to select which features to use.

We use the function `sklearn.svm.SVC` from `scikit-learn` in this problem.

An example on how to use the `SVC` is given in http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC. In short, we do the following (for a linear kernel):
- Instantiate an SVC object: `cls = SVC(kernel='linear')`
- Train the SVM using the feature vector matrix `train_X`, and label vector `train_Y`: `cls.fit(train_X, train_Y)`
- Predict labels on the test set `Test_X` using: `cls.predict(Test_X)`

You can use or adapt the following functions to visualize the SVM decision regions and support vectors in 2D.

In [ ]:
import numpy as np
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC

import warnings
warnings.filterwarnings('ignore')

def Plot_SVM_decision_regions(clf: SVC, data: np.array, labels: np.array) -> None:
    '''
    This function is for plotting the decision area of SVM

    Args:
    - clf: SVM model
    - data: Data with two features
    - labels: Corresponding labels of the data
    '''
    x_min, x_max = data[:,0].min() - 0.2, data[:,0].max() + 0.2
    y_min, y_max = data[:,1].min() - 0.2, data[:,1].max() + 0.2
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.002),np.arange(y_min, y_max, 0.002))
    # we need to use the LabelEncoder here to make contourf work
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(labels)
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = label_encoder.transform(Z)
    Z = Z.reshape(xx.shape)
    #Plotting
    plt.figure(figsize=(10,6))
    sns.scatterplot(x = data[:,0], y = data[:,1],hue=labels)
    plt.contourf(xx, yy, Z, cmap=plt.cm.ocean, alpha=0.2)
    plt.legend()
    plt.title('Decision Area of SVM')
    plt.show()

def Plot_Support_Vectors(clf: SVC, data: np.array) -> None:
    '''
    This function is for plotting the support vectors of the SVM model

    Args:
    - clf: SVM model
    - data: Data with two features
    '''
    x_min, x_max = data[:,0].min() - 0.2, data[:,0].max() + 0.2
    y_min, y_max = data[:,1].min() - 0.2, data[:,1].max() + 0.2
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.002),np.arange(y_min, y_max, 0.002))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    # we need to use the LabelEncoder here to make contourf work
    label_encoder = LabelEncoder()
    Z = label_encoder.fit_transform(Z)
    Z = Z.reshape(xx.shape)
    #Plotting
    plt.figure(figsize=(10,6))
    plt.scatter(clf.support_vectors_[:,0], clf.support_vectors_[:,1], c='k',alpha=0.4,label='support vector')
    plt.contourf(xx, yy, Z, cmap=plt.cm.ocean, alpha=0.2)
    plt.legend()
    plt.title('Support Vectors')
    plt.show()

### Task 10

Create a linear SVM with different penalty terms $C=\{0.1, 1, 10\}$ and compare with the results in Problem 1.

### Task 11

Try different kernels ('rbf', 'poly', 'sigmoid') and compare the results. Choose one of the kernels and use different penalty terms $C$. What happens with the performance on the training set when you increase $C$? What happens with the performance on the test set?